# IBM Applied Data Science Capstone Course by Coursera
### Week 5 Final Report
**_Opening a Shopping Mall in Kiev, Ukraine_**
- Build a dataframe of neighborhoods in in Kiev, Ukraine by web scraping the data from Wikipedia page
- Get the geographical coordinates of the neighborhoods
- Obtain the venue data for the neighborhoods from Foursquare API
- Explore and cluster the neighborhoods
- Select the best cluster to open a new shopping mall
***

### 1. Import libraries

In [164]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder # to get coordinates

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print("Libraries imported.")

Libraries imported.


### 2. Scrap data from Wikipedia page into a DataFrame

In [165]:
# send the GET request
data = requests.get("https://en.wikipedia.org/wiki/Category:Neighborhoods_in_Kiev").text

In [166]:
# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(data, 'html.parser')

In [167]:
# create a list to store neighborhood data
neighborhoodList = []

In [168]:
# append the data into the list
for row in soup.find_all("div", class_="mw-category")[0].findAll("li"):
    neighborhoodList.append(row.text)

In [171]:
# create a new DataFrame from the list
kl_df = pd.DataFrame({"Neighborhood": neighborhoodList})

kl_df.head()

,Neighborhood
0,Subdivisions of Kiev
1,Batyieva Hora
2,Berezniaky
3,Borshchahivka
4,Bortnychi


In [172]:
# print the number of rows of the dataframe
kl_df.shape

(40, 1)

### 3. Get the geographical coordinates

In [173]:
# define a function to get coordinates
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Kiev, Ukraine'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [174]:
# call the function to get the coordinates, store in a new list using list comprehension
coords = [ get_latlng(neighborhood) for neighborhood in kl_df["Neighborhood"].tolist() ]

In [175]:
coords

[[50.50075000000004, 30.454720000000066],
 [50.45056000000005, 30.524280000000033],
 [50.42191992887701, 30.598880076605717],
 [50.441615674026835, 30.432408272763368],
 [50.38489000000004, 30.69542000000007],
 [49.50890000000004, 26.87817000000007],
 [50.41198991993285, 30.666640011108523],
 [50.401401660778674, 30.508507136043484],
 [50.45923000000005, 30.434510000000046],
 [50.314050000000066, 30.56788000000006],
 [50.446840000000066, 30.57630000000006],
 [50.40880429841915, 30.67514151248835],
 [50.45056000000005, 30.524280000000033],
 [50.29997003332557, 30.57517003000735],
 [50.483190000000036, 30.47619000000003],
 [50.46728000000007, 30.62707000000006],
 [50.44870006607415, 30.604580026112217],
 [49.50982000000005, 30.565780000000075],
 [50.442799919499265, 30.528799966674562],
 [50.83442000000008, 28.673290000000065],
 [50.500680000000045, 30.497520000000065],
 [50.45056000000005, 30.524280000000033],
 [50.408660000000054, 30.617770000000064],
 [50.43228461873581, 30.5420310969

In [176]:
# create temporary dataframe to populate the coordinates into Latitude and Longitude
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

In [177]:
# merge the coordinates into the original dataframe
kl_df['Latitude'] = df_coords['Latitude']
kl_df['Longitude'] = df_coords['Longitude']

In [178]:
# check the neighborhoods and the coordinates
print(kl_df.shape)
kl_df

(40, 3)


,Neighborhood,Latitude,Longitude
0,Subdivisions of Kiev,50.500750,30.454720
1,Batyieva Hora,50.450560,30.524280
2,Berezniaky,50.421920,30.598880
3,Borshchahivka,50.441616,30.432408
4,Bortnychi,50.384890,30.695420
5,Cherepanova Hora,49.508900,26.878170
6,Chervonyi Khutir,50.411990,30.666640
7,Demiivka,50.401402,30.508507
8,Feofaniya,50.459230,30.434510
9,Holosiivskyi National Nature Park,50.314050,30.567880


In [179]:
# save the DataFrame as CSV file
kl_df.to_csv("kl_df.csv", index=False)

### 4. Create a map of Kiev with neighborhoods superimposed on top

In [180]:
# get the coordinates of Kiev
address = 'Kiev, Ukraine'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Kiev, Ukraine {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Kiev, Ukraine 50.4500336, 30.5241361.


In [83]:
# create map of Kiev using latitude and longitude values
map_kl = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(kl_df['Latitude'], kl_df['Longitude'], kl_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_kl)  
    
map_kl

In [181]:
# save the map as HTML file
map_kl.save('map_kl.html')

### 5. Use the Foursquare API to explore the neighborhoods

In [204]:
# define Foursquare Credentials and Version
CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 
CLIENT_SECRET:


**Now, let's get the top 100 venues that are within a radius of 2000 meters.**

In [183]:
radius = 2000
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(kl_df['Latitude'], kl_df['Longitude'], kl_df['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [184]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(2845, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Subdivisions of Kiev,50.50075,30.45472,Сільпо,50.505423,30.453150,Supermarket
1,Subdivisions of Kiev,50.50075,30.45472,Сквер возле Мостицкой церкви,50.501991,30.445518,Park
2,Subdivisions of Kiev,50.50075,30.45472,Парк «Березовий Гай»,50.497192,30.455870,Park
3,Subdivisions of Kiev,50.50075,30.45472,Domino's Pizza,50.505778,30.450630,Pizza Place
4,Subdivisions of Kiev,50.50075,30.45472,G13,50.497909,30.465418,Performing Arts Venue


**Let's check how many venues were returned for each neighorhood**

In [185]:
venues_df.groupby(["Neighborhood"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
Batyieva Hora,100,100,100,100,100,100
Berezniaky,100,100,100,100,100,100
Borshchahivka,100,100,100,100,100,100
Bortnychi,21,21,21,21,21,21
Chervonyi Khutir,100,100,100,100,100,100
Demiivka,100,100,100,100,100,100
Feofaniya,100,100,100,100,100,100
Holosiivskyi National Nature Park,13,13,13,13,13,13
Hydropark in Kiev,100,100,100,100,100,100


**Let's find out how many unique categories can be curated from all the returned venues**

In [186]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 291 uniques categories.


In [187]:
# print out the list of categories
venues_df['VenueCategory'].unique()[:50]

array(['Supermarket', 'Park', 'Pizza Place', 'Performing Arts Venue',
       'Hookah Bar', 'Spa', 'Wine Shop', 'Chinese Restaurant',
       'Playground', 'Pet Store', 'Middle Eastern Restaurant',
       'Cosmetics Shop', 'Italian Restaurant', 'Gym', 'Tanning Salon',
       'Toy / Game Store', 'Eastern European Restaurant',
       'Clothing Store', 'Shopping Mall', 'Dance Studio', 'Multiplex',
       'Pool Hall', 'Coffee Shop', 'Café', 'Bubble Tea Shop',
       'Flower Shop', 'Gym / Fitness Center', 'Electronics Store',
       'Candy Store', 'History Museum', 'Fast Food Restaurant',
       'Rest Area', 'Organic Grocery', 'Furniture / Home Store',
       'Big Box Store', 'Auto Garage', 'Yoga Studio', 'Bakery',
       'Farmers Market', 'Gastropub', 'Ice Cream Shop', 'Shoe Store',
       'Sushi Restaurant', 'BBQ Joint', 'Massage Studio',
       'Salon / Barbershop', 'Tennis Stadium', 'Cupcake Shop',
       'Mobile Phone Shop', 'Bank'], dtype=object)

In [188]:
# check if the results contain "Shopping Mall"
"Shopping Mall" in venues_df['VenueCategory'].unique()

True

### 6. Analyze Each Neighborhood

In [189]:
# one hot encoding
kl_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
kl_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [kl_onehot.columns[-1]] + list(kl_onehot.columns[:-1])
kl_onehot = kl_onehot[fixed_columns]

print(kl_onehot.shape)
kl_onehot.head()

(2845, 292)


,Neighborhoods,Accessories Store,Adult Boutique,Airport Service,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Bakery,Bank,Bar,Basketball Court,Bath House,Bay,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boarding House,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Buffet,Burger Joint,Bus Line,Bus Station,Bus Stop,Butcher,Cafeteria,Café,Campground,Canal,Candy Store,Car Wash,Castle,Caucasian Restaurant,Cave,Champagne Bar,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Churrascaria,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Quad,College Theater,Comfort Food Restaurant,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Country Dance Club,Cultural Center,Cupcake Shop,Dairy Store,Dance Studio,Deli / Bodega,Dessert Shop,Diner,Discount Store,Doctor's Office,Dog Run,Doner Restaurant,Drive-in Theater,Eastern European Restaurant,Electronics Store,Event Space,Exhibit,Fabric Shop,Factory,Fair,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Film Studio,Fish Market,Fishing Spot,Fishing Store,Flower Shop,Food,Food & Drink Shop,Food Truck,Forest,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,General Entertainment,Gift Shop,Go Kart Track,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gun Range,Gun Shop,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hookah Bar,Hotel,IT Services,Ice Cream Shop,Indie Movie Theater,Indie Theater,Indoor Play Area,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Lake,Leather Goods Store,Lebanese Restaurant,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monastery,Monument / Landmark,Motorcycle Shop,Mountain,Movie Theater,Multiplex,Museum,Music Store,Music Venue,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Other Repair Shop,Outdoor Sculpture,Outdoor Supply Store,Outlet Store,Paintball Field,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Perfume Shop,Pet Store,Pharmacy,Photography Studio,Pie Shop,Pier,Pizza Place,Playground,Plaza,Pool,Pool Hall,Print Shop,Pub,Public Art,Racecourse,Racetrack,Ramen Restaurant,Record Shop,Recording Studio,Recreation Center,Rental Car Location,Rental Service,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,River,Rock Climbing Spot,Rock Club,Roller Rink,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Sauna / Steam Room,Scenic Lookout,School,Scottish Restaurant,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Ski Area,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,Souvenir Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Squash Court,Stables,Stadium,Steakhouse,Summer Camp,Supermarket,Supplement Shop,Sushi Restaurant,Tanning Salon,Tatar Restaurant,Tea Room,Temple,Tennis Court,Tennis Stadium,Theater,Theme Park,Theme Park Ride / Attraction,Toy / Game Store,Track,Trail,Train Station,Tree,Ukrainian Restaurant,Used Bookstore,Vacation Rental,Vape Store,Varenyky restaurant,Vegetarian / Vegan Restaurant,Veterinarian,Video Game Store,Vietnamese Restaurant,Waterfall,Waterfront,West-Ukrainian Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Subdivisions of Kiev,0,0,0,0,0,0,0,0,0,0,

**Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category**

In [190]:
kl_grouped = kl_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(kl_grouped.shape)
kl_grouped

(36, 292)


,Neighborhoods,Accessories Store,Adult Boutique,Airport Service,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Bakery,Bank,Bar,Basketball Court,Bath House,Bay,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boarding House,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Buffet,Burger Joint,Bus Line,Bus Station,Bus Stop,Butcher,Cafeteria,Café,Campground,Canal,Candy Store,Car Wash,Castle,Caucasian Restaurant,Cave,Champagne Bar,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Churrascaria,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Quad,College Theater,Comfort Food Restaurant,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Country Dance Club,Cultural Center,Cupcake Shop,Dairy Store,Dance Studio,Deli / Bodega,Dessert Shop,Diner,Discount Store,Doctor's Office,Dog Run,Doner Restaurant,Drive-in Theater,Eastern European Restaurant,Electronics Store,Event Space,Exhibit,Fabric Shop,Factory,Fair,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Film Studio,Fish Market,Fishing Spot,Fishing Store,Flower Shop,Food,Food & Drink Shop,Food Truck,Forest,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,General Entertainment,Gift Shop,Go Kart Track,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gun Range,Gun Shop,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hookah Bar,Hotel,IT Services,Ice Cream Shop,Indie Movie Theater,Indie Theater,Indoor Play Area,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Lake,Leather Goods Store,Lebanese Restaurant,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monastery,Monument / Landmark,Motorcycle Shop,Mountain,Movie Theater,Multiplex,Museum,Music Store,Music Venue,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Other Repair Shop,Outdoor Sculpture,Outdoor Supply Store,Outlet Store,Paintball Field,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Perfume Shop,Pet Store,Pharmacy,Photography Studio,Pie Shop,Pier,Pizza Place,Playground,Plaza,Pool,Pool Hall,Print Shop,Pub,Public Art,Racecourse,Racetrack,Ramen Restaurant,Record Shop,Recording Studio,Recreation Center,Rental Car Location,Rental Service,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,River,Rock Climbing Spot,Rock Club,Roller Rink,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Sauna / Steam Room,Scenic Lookout,School,Scottish Restaurant,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Ski Area,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,Souvenir Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Squash Court,Stables,Stadium,Steakhouse,Summer Camp,Supermarket,Supplement Shop,Sushi Restaurant,Tanning Salon,Tatar Restaurant,Tea Room,Temple,Tennis Court,Tennis Stadium,Theater,Theme Park,Theme Park Ride / Attraction,Toy / Game Store,Track,Trail,Train Station,Tree,Ukrainian Restaurant,Used Bookstore,Vacation Rental,Vape Store,Varenyky restaurant,Vegetarian / Vegan Restaurant,Veterinarian,Video Game Store,Vietnamese Restaurant,Waterfall,Waterfront,West-Ukrainian Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Batyieva Hora,0.01,0.00,0.000000,0.00,0.0

In [191]:
len(kl_grouped[kl_grouped["Shopping Mall"] > 0])

10

**Create a new DataFrame for Shopping Mall data only**

In [192]:
kl_mall = kl_grouped[["Neighborhoods","Shopping Mall"]]

In [193]:
kl_mall.head()

,Neighborhoods,Shopping Mall
0,Batyieva Hora,0.00
1,Berezniaky,0.00
2,Borshchahivka,0.00
3,Bortnychi,0.00
4,Chervonyi Khutir,0.01


### 7. Cluster Neighborhoods
Run k-means to cluster the neighborhoods in Kiev into 4 clusters.

In [194]:
pip install geocoder 

Note: you may need to restart the kernel to use updated packages.


In [195]:
# set number of clusters
kclusters = 3

kl_clustering = kl_mall.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(kl_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:7] 

array([0, 0, 0, 0, 1, 1, 0])

In [196]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
kl_merged = kl_mall.copy()

# add clustering labels
kl_merged["Cluster Labels"] = kmeans.labels_

In [197]:
kl_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
kl_merged.head()

,Neighborhood,Shopping Mall,Cluster Labels
0,Batyieva Hora,0.00,0
1,Berezniaky,0.00,0
2,Borshchahivka,0.00,0
3,Bortnychi,0.00,0
4,Chervonyi Khutir,0.01,1


In [198]:
# merge kiev_grouped with kiev_data to add latitude/longitude for each neighborhood
kl_merged = kl_merged.join(kl_df.set_index("Neighborhood"), on="Neighborhood")


print(kl_merged.shape)
kl_merged.head() # check the last columns!

(36, 5)


,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
0,Batyieva Hora,0.00,0,50.450560,30.524280
1,Berezniaky,0.00,0,50.421920,30.598880
2,Borshchahivka,0.00,0,50.441616,30.432408
3,Bortnychi,0.00,0,50.384890,30.695420
4,Chervonyi Khutir,0.01,1,50.411990,30.666640


In [199]:
# sort the results by Cluster Labels
print(kl_merged.shape)
kl_merged.sort_values(["Cluster Labels"], inplace=True)
kl_merged

(36, 5)


,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
0,Batyieva Hora,0.000000,0,50.450560,30.524280
33,Vydubychi,0.000000,0,50.389600,30.560090
32,Voskresenka (Kiev),0.000000,0,50.468431,30.601302
31,Vidradnyi,0.000000,0,50.376550,30.698410
29,Teremky,0.000000,0,50.367510,30.455120
27,Rybalskyi Peninsula,0.000000,0,50.345847,30.620486
26,Rusanivka,0.000000,0,50.439367,30.591960
25,Pyrohiv,0.000000,0,50.353390,30.503610
24,Pushcha-Vodytsia,0.000000,0,50.534550,30.355890
23,Protasiv Yar,0.000000,0,50.424430,30.498870


**Finally, let's visualize the resulting clusters**

In [200]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(kl_merged['Latitude'], kl_merged['Longitude'], kl_merged['Neighborhood'], kl_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=10,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [144]:
# save the map as HTML file
map_clusters.save('map_clusters.html')

### 8. Examine Clusters

#### Cluster 0

In [201]:
df_c_0 = kl_merged.loc[kl_merged['Cluster Labels'] == 0]
print(df_c_0.shape)
df_c_0

(26, 5)


,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
0,Batyieva Hora,0.0,0,50.450560,30.524280
33,Vydubychi,0.0,0,50.389600,30.560090
32,Voskresenka (Kiev),0.0,0,50.468431,30.601302
31,Vidradnyi,0.0,0,50.376550,30.698410
29,Teremky,0.0,0,50.367510,30.455120
27,Rybalskyi Peninsula,0.0,0,50.345847,30.620486
26,Rusanivka,0.0,0,50.439367,30.591960
25,Pyrohiv,0.0,0,50.353390,30.503610
24,Pushcha-Vodytsia,0.0,0,50.534550,30.355890
23,Protasiv Yar,0.0,0,50.424430,30.498870


#### Cluster 1

In [202]:
df_c_1 = kl_merged.loc[kl_merged['Cluster Labels'] == 1]
print(df_c_1.shape)
df_c_1

(8, 5)


,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
20,"Petrivka, Kiev",0.010000,1,50.489280,30.492460
5,Demiivka,0.010000,1,50.401402,30.508507
28,Subdivisions of Kiev,0.010000,1,50.500750,30.454720
4,Chervonyi Khutir,0.010000,1,50.411990,30.666640
30,Troieshchyna,0.010309,1,50.510750,30.606540
18,Osokorky,0.010000,1,50.408660,30.617770
16,"Obolon, Kiev",0.010000,1,50.500680,30.497520
13,Lisovyi masyv,0.010000,1,50.467280,30.627070


#### Cluster 2

In [203]:
df_c_2 = kl_merged.loc[kl_merged['Cluster Labels'] == 2]
print(df_c_2.shape)
df_c_2

(2, 5)


,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
9,Kharkivskyi Masyv,0.011364,2,50.408804,30.675142
34,Vynohradar,0.010989,2,50.513930,30.424390


#### Observations:
Most of the shopping centers are concentrated in cluster 0 and moderate number in cluster 1. On the other hand, cluster 2 has very low number to totally no shopping mall in the neighborhoods. This represents a great opportunity and high potential areas to open new shopping malls as there is very little to no competition from existing malls. Meanwhile, shopping malls in cluster 0 are likely suffering from intense competition due to oversupply and high concentration of shopping malls. From another perspective, this also shows that the oversupply of shopping malls mostly happened in the central area of the city, with the suburb area still have very few shopping malls. Therefore, this project recommends property developers to capitalize on these findings to open new shopping malls in neighborhoods in cluster 2 with little to no competition. Property developers with unique selling propositions to stand out from the competition can also open new shopping malls in neighborhoods in cluster 1 with moderate competition. Lastly, property developers are advised to avoid neighborhoods in cluster 0 which already have high concentration of shopping malls and suffering from intense competition.